In [18]:
from neo4j import GraphDatabase,basic_auth
from random import randint,choice
import datetime

In [19]:
#Create Connection to Neo4j
#conn = GraphDatabase.driver(uri = "bolt://100.26.227.192:38697",auth=basic_auth("neo4j","frames-crashes-wind"))
conn = GraphDatabase.driver(uri = "bolt://3.85.125.165:7687",auth=basic_auth("neo4j","holddowns-islands-thumb"))

In [20]:
conn.address

IPv4Address(('3.85.125.165', 7687))

In [21]:
conn.session()

In [22]:
conn.verify_connectivity()

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: ExperimentalWarning: The configuration may change in the future.
  """Entry point for launching an IPython kernel.


'Neo4j/4.2.3'

In [23]:
sess = conn.session()

In [24]:
sess.run(query="CREATE(a:Product{ProdName:'Pen'})")
sess.run("CREATE(a:Product{ProdName:'Printer'})")
sess.run("CREATE(s:Supplier{SuppName :'Reynolds'})")
sess.run("CREATE(s:Supplier{SuppName :'Brother'})")

sess.run("MATCH(a:Product{ProdName:'Pen'}) \
         MATCH(s:Supplier{SuppName :'Reynolds'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MATCH(a:Product{ProdName:'Printer'}) \
         MATCH(s:Supplier{SuppName :'Brother'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MATCH(a:Product{ProdName:'Printer'}) \
         MERGE(s:Supplier{SuppName :'Cannon'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MATCH(a:Product{ProdName:'Printer'}) \
         MERGE(s:Supplier{SuppName :'HP'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MERGE(a:Product{ProdName:'File Cover'}) \
         MERGE(s:Supplier{SuppName :'up&up'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MERGE(a:Product{ProdName:'Envelopes'}) \
         MERGE(s:Supplier{SuppName :'Prince'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MERGE(a:Product{ProdName:'Markers'}) \
         MERGE(s:Supplier{SuppName :'Reynolds'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")


In [25]:
sess.run("Create(c:Category{CategoryNm : 'Stationary'})")
sess.run("MATCH(c:Category) \
          MATCH (p:Product)\
          Create (p)-[:HAS_CATEGORY]->(c)\
          "
        )

In [9]:
prod_list = ['Printer','Pen','Markers','File Cover','Notebooks','Envelopes']
cust_list =['WF','BoA','JPM','Citi','USB','5/3rd','FirstCitizen','Walmart','Kroger','HT','CTS','TIAA']

In [26]:
sess.run("CREATE(a:Customer{CompanyName:'WF'})")
sess.run("CREATE(a:Customer{CompanyName:'BoA'})")
sess.run("CREATE(a:Customer{CompanyName:'JPM'})")
sess.run("CREATE(a:Customer{CompanyName:'Citi'})")
sess.run("CREATE(a:Customer{CompanyName:'USB'})")
sess.run("CREATE(a:Customer{CompanyName:'5/3rd'})")
sess.run("CREATE(a:Customer{CompanyName:'FirstCitizen'})")
sess.run("CREATE(a:Customer{CompanyName:'Walmart'})")
sess.run("CREATE(a:Customer{CompanyName:'Kroger'})")
sess.run("CREATE(a:Customer{CompanyName:'HT'})")
sess.run("CREATE(a:Customer{CompanyName:'CTS'})")
sess.run("CREATE(a:Customer{CompanyName:'TIAA'})")

In [27]:
%%time
for order in range(100001,100031):
    #print('Order_id : ' + str(order))
    prod=choice(prod_list)
    #print(prod)
    
    start_date = datetime.date(2020, 1, 1)
    random_days = randint(1,364)
    Order_date = start_date + datetime.timedelta(days=random_days)
    #print(Order_date)
    
    qty=randint(1,100)
    #print(qty)
    
    CompanyName=choice(cust_list)
    
    
    line_item = randint(1,4)
    if line_item==1:
        #Insert Data into Graph
        sess.run("MATCH (p:Product{ProdName : $prod})\
                  MATCH (c:Customer{CompanyName : $CompanyName})\
                  MERGE(o:Order{OrderId:$order,OrderDt:$Order_date }) \
                  MERGE (c)-[:PLACED_ORDER]->(o)-[:HAS_PROD{Qty:$qty}]->(p)\
                 ",
                 prod=prod,
                 order=order,
                 Order_date=Order_date,
                 CompanyName=CompanyName,
                 qty=qty
            )
        print(str(order) +' '+CompanyName +' '+prod+' '+str(Order_date)+' '+str(qty))
    else:
        #Insert Data into Graph
        sess.run("MATCH (c:Customer{CompanyName : $CompanyName})\
                  MERGE(o:Order{OrderId:$order,OrderDt:$Order_date }) \
                  MERGE (c)-[:PLACED_ORDER]->(o)\
                 ",                 
                 order=order,
                 Order_date=Order_date,
                 CompanyName=CompanyName
                )
        prod_tmp_lst =[]
        for i in range(1,line_item+1):
            prod=choice(prod_list)
            if prod in prod_tmp_lst:
                continue
            prod_tmp_lst.append(prod)
            qty=randint(1,100)
            #Insert Data into Graph
            sess.run("MATCH (p:Product{ProdName : $prod})\
                      MATCH (o:Order{OrderId:$order }) \
                      MERGE (o)-[:HAS_PROD{Qty:$qty}]->(p)\
                     ",
                     prod=prod,
                     order=order,
                     qty=qty
                )
            print('ELSE : Line_Item : '+str(i)+' '+str(order) +' '+CompanyName+' '+str(Order_date) +' '+prod+' '+str(qty))
    print('\n')

ELSE : Line_Item : 1 100001 TIAA 2020-12-02 Envelopes 96
ELSE : Line_Item : 3 100001 TIAA 2020-12-02 File Cover 26
ELSE : Line_Item : 4 100001 TIAA 2020-12-02 Markers 50


100002 CTS Markers 2020-03-23 81


ELSE : Line_Item : 1 100003 CTS 2020-06-06 Notebooks 5
ELSE : Line_Item : 2 100003 CTS 2020-06-06 Markers 86
ELSE : Line_Item : 3 100003 CTS 2020-06-06 Printer 69
ELSE : Line_Item : 4 100003 CTS 2020-06-06 Envelopes 10


ELSE : Line_Item : 1 100004 Citi 2020-11-06 Notebooks 54
ELSE : Line_Item : 2 100004 Citi 2020-11-06 Envelopes 3
ELSE : Line_Item : 3 100004 Citi 2020-11-06 File Cover 86


ELSE : Line_Item : 1 100005 BoA 2020-02-15 Pen 38
ELSE : Line_Item : 2 100005 BoA 2020-02-15 Printer 61


ELSE : Line_Item : 1 100006 FirstCitizen 2020-09-26 Envelopes 23
ELSE : Line_Item : 2 100006 FirstCitizen 2020-09-26 Printer 20


ELSE : Line_Item : 1 100007 5/3rd 2020-12-26 Notebooks 18
ELSE : Line_Item : 3 100007 5/3rd 2020-12-26 Markers 5


ELSE : Line_Item : 1 100008 USB 2020-03-20 Noteb